In [1]:
import pandas as pd

In [18]:
actions_df = pd.read_csv('../datasets/r3.1-actions_df_1.csv')
test_actions_df = pd.read_csv('../datasets/r3.2-actions_df_1.csv')

In [20]:
filtered_ids = set(actions_df['id'])
filtered_test_ids = set(test_actions_df['id'])

In [9]:
len(filtered_ids)

8317

In [10]:
train_df = pd.read_csv('../datasets/r3.1.csv')
train_df = train_df[train_df.apply(lambda x: x['id'] in filtered_ids, axis=1)]

/tmp/ipykernel_7483/699087821.py:1: DtypeWarning: Columns (3,4,9,10,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../datasets/r3.1.csv')


In [29]:
test_df = pd.read_csv('../datasets/r3.2.csv')
test_df = test_df[test_df.apply(lambda x: x['id'] in filtered_test_ids, axis=1)]

/tmp/ipykernel_7483/808152523.py:1: DtypeWarning: Columns (3,4,9,10,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('../datasets/r3.2.csv')


In [30]:
train_df.shape, test_df.shape

((8317, 23), (8435, 23))

In [33]:
def drop_and_split(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    X = (
        df
        .drop('date', axis=1)
        .drop('filename', axis=1)
        .drop('content', axis=1)
        .drop('activity', axis=1)
        .drop('pc', axis=1)
        .drop('user', axis=1)
        .drop('id', axis=1)
        .drop('target', axis=1)
    )
    y = df['target']
    return X, y


X_train, y_train = drop_and_split(train_df)
X_test, y_test = drop_and_split(test_df)

In [61]:
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, class_weight={True: 1000, False: 1})
rfc.fit(X_train_scaled, y_train)
display(rfc.score(X_train_scaled, y_train))

1.0

In [62]:
y_pred = rfc.predict(X_test_scaled)

from sklearn.metrics import classification_report, f1_score, accuracy_score

print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred))


              precision    recall  f1-score   support

       False       1.00      1.00      1.00      8413
        True       0.00      0.00      0.00        22

    accuracy                           1.00      8435
   macro avg       0.50      0.50      0.50      8435
weighted avg       0.99      1.00      1.00      8435

0.0


/home/alexeyshesh/insider-threat-detection/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/alexeyshesh/insider-threat-detection/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/alexeyshesh/insider-threat-detection/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

In [63]:
set(y_pred)

{False}

In [66]:
from imblearn.ensemble import BalancedRandomForestClassifier

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

brf = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
brf.fit(X_train_scaled, y_train)
display(brf.score(X_train_scaled, y_train))

/home/alexeyshesh/insider-threat-detection/.venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/alexeyshesh/insider-threat-detection/.venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/alexeyshesh/insider-threat-detection/.venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the orig

0.953589034507635

In [67]:
y_pred = brf.predict(X_test_scaled)

from sklearn.metrics import classification_report, f1_score, accuracy_score

print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred))

              precision    recall  f1-score   support

       False       1.00      0.93      0.96      8413
        True       0.00      0.00      0.00        22

    accuracy                           0.93      8435
   macro avg       0.50      0.46      0.48      8435
weighted avg       0.99      0.93      0.96      8435

0.0


In [69]:
from collections import Counter
Counter(y_pred)

Counter({False: 7826, True: 609})

In [70]:
train_df[train_df['target'] == 1]

,id,date,user,Connect,Disconnect,is_new_pc,avg_daily_device_connection,device_connection_num,device_con_above_avg,pc,...,Logoff,Logon,filename,content,avg_daily_file_usage,file_usage_num,file_above_avg,before_avg_logon,after_avg_logout,target
158491,{V1J9-P7JR21ZY-0017JSES},07/01/2010 02:23:04,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1178.678082,0.00000,True
158492,{Q7Q0-F7FL92IO-3645ARQH},07/01/2010 03:53:50,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,419.62585,True
158493,{O4Y3-X5SF91ZE-7867MGPF},07/01/2010 04:09:15,CSF0929,True,False,False,1.8,3.0,1.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.678082,0.00000,True
158494,{V6B0-T8YP88MD-6528ZNPI},07/01/2010 05:50:23,CSF0929,False,True,False,1.8,4.0,2.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,241.62585,True
160798,{F9X7-E1CP85QD-3175HSBP},07/02/2010 21:57:07,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,677.62585,True
160800,{V2Q2-P1JW27KX-4514LWVQ},07/02/2010 22:40:25,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.678082,0.00000,True
163567,{O5Y9-E5LL15ZL-7586ILYR},07/08/2010 06:18:28,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1474.678082,0.00000,True
163568,{M0A0-U5AK71ET-3193XUDJ},07/08/2010 06:46:19,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,2.62585,True
164726,{N6J8-T9UV76LX-9928WBKB},07/09/2010 01:07:57,CSF0929,True,False,False,1.8,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2139.678082,0.00000,True
164728,{T0R8-Z6OF30MJ-4136OHNN},07/09/2010 02:51:26,CSF0929,False,True,False,1.8,2.0,0.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,298.62585,True


In [ ]:
'''
ничего не получается

почему?

плохие признаки

у меня есть уже сразу сомнительные действия – они сделаны в нестандартное время

может быть, надо использовать поиск аномалий для поиска сессий с нестандартным временем входа
а потом уже для нестандартного использования файлов
а потом можно пересечь два множества
'''